In [144]:
import qiskit as qiskit
import numpy as np
from random import randint
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit import IBMQ
IBMQ.save_account('7e245f54848bdbcc6bedd42fcafcd2fbe8f81b765b2537e32d39f812c3ccc2e9c755a6ac3e3edc7529982f02954bff4b84cba76cef7fe71928b9f01b092feedf')
IBMQ.load_account()
from qiskit.providers.aer.noise import NoiseModel

configrc.store_credentials:WARNING:2020-06-25 22:53:12,480: Credentials already present. Set overwrite=True to overwrite.
ibmqfactory.load_account:WARNING:2020-06-25 22:53:12,792: Credentials are already in use. The existing account in the session will be replaced.


# Creating Random Circuits

We want to create random circuits to train and validate our neural network on. To do this, we utilize random number generators with a limit on the circuit depth of the largest quantum computer available to us from the IBMQ Experience, 15 qubits. We want two things from this function:
1. Output a QuantumCircuit object that we can later run on multiple backends
2. Output the data in a way that our neural network can understand:
    * A list which has the following Dimensions: number of qubits x circuit depth
    * Each element in the list will correspond to a gate given by the dictionary `operators`

In [192]:
def create_circuit():
    """This function takes no inputs and outputs a random circuit in neural network representation 
    as well as a quantum circuit object"""
    num_qubits = randint(2,15) #Set the circuit width
    len_circuit = randint(10,25) #Set the circuit depth
    circ = np.zeros((num_qubits,len_circuit)) #Initialize the circuit representation for the Neural Net
    qc = qiskit.QuantumCircuit(num_qubits) #Initialize the actual quantum circuit
    operators = {0:qc.id,1:qc.x,2:qc.y,3:qc.z,4:qc.h,5:qc.cx,6:qc.swap} #Define the operators and their corresponding Neural Net representations
    for i in range(len_circuit):
        ctrl = randint(0,num_qubits-1) #Choose a qubit for a single qubit gat to act on or a ctrl qubit for a 2 qubit gate to act on
        targ = randint(0,num_qubits-1) #Choose a target qubit for a 2 qubit gate to act on
        gate_num = randint(1,6) #Choose a gate to implement
        while ctrl == targ: #Make sure the target and ctrl qubit are different
            targ = randint(0,num_qubits-1)
        try:
            #Implement the gate if it is a single qubit gate and add it to the neural net representation
            operators.get(gate_num)(ctrl) 
            circ[ctrl][i] = gate_num
        except:
            try:
                #Implement the gate if it is a 2 qubit gate and add it to the neural net representation
                operators.get(gate_num)(ctrl,targ)
                circ[ctrl][i] = gate_num
                circ[targ][i] = gate_num
            except:
                gate_num = randint(1,6)
    qc.measure_all()
    return circ, qc

In [193]:
def list_of_circuits(num_of_circuits):
    circuits = [0]*num_of_circuits #initialize list by number of desired circuits
    circuit_arrays = [0]*num_of_circuits #initialize list by number of desired circuits

    for i in range(num_of_circuits): 
        #calls create_circuits function desired number of times and puts objects into the arrays
        circuit_arrays[i], circuits[i] = create_circuit()
    return circuit_arrays, circuits

In [201]:

circuit_arrays, circuits = list_of_circuits(1)
list_of_backends = IBMQ.get_provider('ibm-q').backends()
list_of_backends.remove(IBMQ.get_provider('ibm-q').get_backend('ibmq_qasm_simulator'))
simulator = Aer.get_backend('statevector_simulator')

validation_set = np.zeros((len(circuits),len(list_of_backends)))

for i in range(len(circuits)):
    previous_fidelity = 0
    best_backend = 0;
    for j in range(len(list_of_backends)):
        if np.shape(circuit_arrays[i])[0] < list_of_backends[j].configuration().num_qubits:
            coupling_map = list_of_backends[j].configuration().coupling_map
            basis_gates = list_of_backends[j].configuration().basis_gates
            noise_model = NoiseModel.from_backend(list_of_backends[j])
            psi_0 = execute(circuits[i], simulator, shots = 1000, coupling_map = coupling_map, basis_gates = basis_gates).result().get_statevector()
            psi_1 = execute(circuits[i], simulator, shots = 1000, coupling_map = coupling_map, basis_gates = basis_gates, noise_model = noise_model).result().get_statevector()
            fidelity = qiskit.quantum_info.state_fidelity(psi_0, psi_1, validate=True)
            if fidelity > previous_fidelity:
                best_backend = j
                previous_fidelity = fidelity
    validation_set[i, best_backend] = 1
        
        
        

In [154]:
list_of_backends = IBMQ.get_provider('ibm-q').backends()
list_of_backends.remove(IBMQ.get_provider('ibm-q').get_backend('ibmq_qasm_simulator'))
list_of_backends.remove(IBMQ.get_provider('ibm-q').get_backend('ibmq_16_melbourne'))
list_of_backends.remove(IBMQ.get_provider('ibm-q').get_backend('ibmq_vigo'))

In [202]:
validation_set

array([[1., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [205]:
circuit_arrays, circuit = list_of_circuits(1)
simulator = Aer.get_backend('statevector_simulator')
device = IBMQ.get_provider('ibm-q').get_backend('ibmq_london')
coupling_map = device.configuration().coupling_map
basis_gates = device.configuration().basis_gates
noise_model = NoiseModel.from_backend(device)

psi_0 = execute(circuit, simulator, coupling_map = coupling_map, basis_gates = basis_gates).result().get_statevector()

job2 = execute(circuit, simulator, shots = 10, coupling_map = coupling_map, basis_gates = basis_gates, noise_model = noise_model).result().get_statevector()
fidelity = qiskit.quantum_info.state_fidelity(psi_0, job2, validate=True)

Device reported a gate error parameter greater than maximum allowed value (1.000000 > 0.800000). Truncating to maximum value.
Device model returned a depolarizing error parameter greater than maximum allowed value (1.067405 > 1.066667). Truncating to maximum value.
Device reported a gate error parameter greater than maximum allowed value (1.000000 > 0.800000). Truncating to maximum value.
Device model returned a depolarizing error parameter greater than maximum allowed value (1.067339 > 1.066667). Truncating to maximum value.


In [191]:
print(fidelity)

0.0


In [130]:
circuit_arrays, circuit = list_of_circuits(1)
device = IBMQ.get_provider('ibm-q').get_backend('ibmq_london')
job = execute(circuit, device, shots = 1000).result()


In [ ]:
statevector = [value/1000 for value in job.get_counts().values()]


ERROR! Session/line number was not unique in database. History logging moved to new session 47


NameError: name 'state_fidelity' is not defined

In [135]:
np.asarray(statevector)

array([0.031, 0.503, 0.431, 0.035])

In [134]:
job.get_counts()

{'01': 31, '10': 503, '11': 431, '00': 35}

In [207]:
randint(1,15)

5